# Testing File

## Import Library

In [7]:
import tensorflow as tf
import requests
import json
import base64

## Preprocces Input

In [8]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, 'utf-8')]
        ),
    )

def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result,
    })

## Predict

In [9]:
def predict(input):
	response = requests.post(
		'https://system-development-and-operation.up.railway.app/v1/models/hate-speech-model:predict',
		data=prepare_json(input)
	)

	pred = response.json()['predictions'][0][0]
	return pred

text = "Hispanic neighborhoods are all the same; they're just filled with gangs and crime."
result = predict({'Text': text})
print(f'{"Offensive" if round(result) == 1 else "Normal"}')

Offensive
